In [2]:
import pandas as pd
import glob, os    
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
datapath="/Users/soni6/Box/Study/Project/Case Studies/dataset"
demanddata = pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "DeliveryData/DeliveryData*.csv"))))
demanddata.drop(columns=["Unnamed: 0"],inplace=True)
demanddata.columns
# zip
zip_codes=pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "ZIP CODE.csv"))))
zip_codes=list(zip_codes.columns)
zip_codes=[int(i) for i in zip_codes]

In [66]:
zipdemand=demanddata.groupby(['customer ZIP','scenario']).agg({'quantity':'sum'})
zipdemand.reset_index(inplace=True)

In [ ]:
for zipcode in zip_codes[:20]:    
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    zip_index=zipdemand['customer ZIP']==zipcode
    plt.hist(zipdemand[zip_index]['quantity'],bins=30,label=zipcode)
    plt.xlabel('Demand')
    plt.ylabel('Number of scenarios')
    plt.legend()


In [ ]:
for zipcode in zip_codes[:20]:    
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    zip_index=zipdemand['customer ZIP']==zipcode
    sns.lineplot(data=zipdemand[zip_index], x="scenario", y="quantity",label=zipcode)

In [8]:
# commodity demand
commodity_demand=demanddata.groupby(['origin facility','customer ZIP','scenario']).agg({'quantity':'sum'})
commodity_demand.reset_index(inplace=True)


,origin facility,customer ZIP,scenario,quantity
0,ONode0,90001,0,80
1,ONode0,90001,1,78
2,ONode0,90001,2,79
3,ONode0,90001,3,50
4,ONode0,90001,4,77


In [ ]:
# o-d demand analysis: normal and exponential
import numpy as np
for zipcode in zip_codes:    
    for ori_node in set(commodity_demand['origin facility']):
        od_demand= commodity_demand.loc[np.logical_and(commodity_demand['customer ZIP'] == zipcode, commodity_demand['origin facility'] == ori_node)]
        fig = plt.figure(figsize=(10, 6)) #Create figure space
        name=str(ori_node)+ "-" + str(zipcode)
        plt.hist(od_demand['quantity'],bins=30,label=name)
        plt.xlabel('Demand')
        plt.ylabel('Number of scenarios')
        plt.legend()
        plt.savefig('input analysis/'+name+'.png')
        plt.close(fig)


In [ ]:
# demand analysis with scenario: seasonality-no conclusion
for zipcode in zip_codes[:20]:    
    od_demand= commodity_demand.loc[np.logical_and(commodity_demand['customer ZIP'] == zipcode, commodity_demand['origin facility'] == ori_node)]
    name=str(ori_node)+ "-" + str(zipcode)
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    sns.lineplot(data=od_demand, x="scenario", y="quantity",label=name)